In [ ]:
import Pkg; Pkg.activate(".")
using FRESCO
import FUSE, QED, IMAS, VacuumFields
using Plots, ProgressMeter

In [ ]:
recycle = true
if recycle
    dd = IMAS.json2imas("./dd_iter.json")
else
    dd, ini, act = FUSE.init(:ITER, init_from=:ods);
    FUSE.ActorEquilibrium(dd, act; ip_from=:pulse_schedule);
    IMAS.imas2json(dd, "./dd_iter.json");
end;

In [ ]:
canvas0 = FRESCO.Canvas(dd, 65);
profile0 = FRESCO.PprimeFFprime(dd);
eqt = dd.equilibrium.time_slice[];
cp1d = dd.core_profiles.profiles_1d[];
Q0 = FUSE.qed_init_from_imas(eqt, cp1d; uniform_rho = 101);
η0 = FUSE.η_imas(cp1d);

In [ ]:
# Solve equilibrium with shape control to get self-consistent coil currents
canvas, profile = deepcopy(canvas0), deepcopy(profile0);
control = :shape;
# Keep currents fixed in ITER OH coils (1:6), VS coils(13:14), and vacuum vessel(15:162)
fixed_coils = vcat(1:6, 13:14, 15:162);
@time FRESCO.solve!(canvas, profile, 100, 3; control, relax=0.25, tolerance=1e-12, debug=true, initialize_current=true, fixed_coils);
plot(canvas, size=(500,300))

In [ ]:
# SAVE AT THIS POINT
canvas_save = deepcopy(canvas);
dd_save = deepcopy(dd);
Q_save = deepcopy(Q0);

In [ ]:
# Perturb the VS3U coil current by 1 kA (starts at 0)
canvas1, profile1 = deepcopy(canvas), deepcopy(profile);
dI_13 = 1e3;
VacuumFields.set_current!(canvas1.coils[13], dI_13);
@time FRESCO.solve!(canvas1, profile1, 100, 3; relax=0.25, tolerance=1e-6, debug=true, control=:eddy, initialize_current=false, initialize_mutuals=true);

In [ ]:
println("Shift = $(round(canvas1.Zaxis - canvas.Zaxis, digits=5)) m")
p1 = plot(canvas, xrange=(1,13), yrange=(-8.5,8.5), title="Initial", clims=(-160,160))
p2 = plot(canvas1, xrange=(1,13), yrange=(-8.5,8.5), title="Shifted with Eddys", clims=(-160,160), ylabel="")
plot(p1,p2, size=(800,450),left_margin=4Plots.mm)

In [ ]:
# This creates a QED Build struct based on a FRESCO canvas.coils
# The voltage waveforms are set to be constant values that match V=IR
function Build(canvas, dd)

    coils = canvas.coils
    Mcc = [VacuumFields.mutual(c1, c2) for c1 in coils, c2 in coils]
    Ic = [VacuumFields.current(c) / VacuumFields.turns(c) for c in coils] #current per turn
    Rc = [VacuumFields.resistance(c) for c in coils];
    Vc = Ic .* Rc
    V_waveforms = [QED.Waveform(Vc[k]) for k in eachindex(Vc)]
    
    eqt = dd.equilibrium.time_slice[]
    cp1d = dd.core_profiles.profiles_1d[]
    Ip = eqt.global_quantities.ip
    
    # COIL MUTUALS
    #image = VacuumFields.Image(eqt)
    #Mpc = [VacuumFields.mutual(image, coil, Ip) for coil in coils]
    Mpc = [-FRESCO.plasma_flux_at_coil(k, canvas) / Ip for k in eachindex(coils)]
    dMpc_dt = zero(Mpc) # How Mpc changes in time (like shape)... to test later
    
    # INTERNAL INDUCTANCE
    It = IMAS.cumtrapz(cp1d.grid.area, cp1d.j_tor)
    Wp = 0.5 * IMAS.trapz(cp1d.grid.psi, It)
    Li = 2 * Wp / Ip^2
    
    # EXTERNAL INDUCTANCE
    ψb = eqt.profiles_1d.psi[end]
    ψc = sum(Mpc[k] * Ic[k] for k in eachindex(coils))
    Le = (ψb - ψc) / Ip
    Lp = Li + Le

    Pohm = dd.core_sources.source[:ohmic].profiles_1d[].electrons.power_inside[end]
    Ini = dd.core_profiles.global_quantities.current_non_inductive[]
    Iohm = Ip - Ini
    Rp = Pohm / (Ip * Iohm)

    Vni = Rp * Ini
    
    return QED.QED_build(Ic, Vc, Rc, Mcc, Vni, Rp, Lp, Mpc, dMpc_dt, V_waveforms);
end

function update_build!(build, canvas, dd)
    coils = canvas.coils

    eqt = dd.equilibrium.time_slice[]
    cp1d = dd.core_profiles.profiles_1d[]
    Ip = eqt.global_quantities.ip
    #image = VacuumFields.Image(eqt)    
    
    for (k, coil) in enumerate(coils)
        build.Ic[k] =  VacuumFields.current(coil) / VacuumFields.turns(coil)
        #build.Mpc[k] = VacuumFields.mutual(image, coil, Ip)
        build.Mpc[k] = FRESCO.plasma_flux_at_coil(k, canvas) / Ip
    end
    
    # INTERNAL INDUCTANCE
    It = IMAS.cumtrapz(cp1d.grid.area, cp1d.j_tor)
    Wp = 0.5 * IMAS.trapz(cp1d.grid.psi, It)
    Li = 2 * Wp / Ip^2
    
    # EXTERNAL INDUCTANCE
    ψb = eqt.profiles_1d.psi[end]
    ψc = sum(build.Mpc[k] * build.Ic[k] for k in eachindex(coils))
    Le = (ψb - ψc) / Ip
    build.Lp = Li + Le

    Pohm = dd.core_sources.source[:ohmic].profiles_1d[].electrons.power_inside[end]
    Ini = dd.core_profiles.global_quantities.current_non_inductive[]
    Iohm = Ip - Ini
    build.Rp = Pohm / (Ip * Iohm)

    build.Vni = build.Rp * Ini
    
    return build
end

In [ ]:
function update_from_fresco!(dd, canvas, profile)
    eq = dd.equilibrium
    eqt = eq.time_slice[]
    eqt1d = eqt.profiles_1d
    eq2d = resize!(eqt.profiles_2d, 1)[1]

    eqt.global_quantities.magnetic_axis.r = canvas.Raxis
    eqt.global_quantities.magnetic_axis.z = canvas.Zaxis
    eqt.global_quantities.psi_boundary = canvas.Ψbnd
    eqt.global_quantities.psi_axis = canvas.Ψaxis

    Npsi = length(eqt1d.psi)
    eqt1d.psi = range(canvas.Ψaxis, canvas.Ψbnd, Npsi)
    # p' doesn't change
    eqt1d.f_df_dpsi = profile.ffprime.(eqt1d.psi_norm) .* profile.ffp_scale

    fend = eq.vacuum_toroidal_field.b0[end] * eq.vacuum_toroidal_field.r0
    f2 = 2 * IMAS.cumtrapz(eqt1d.psi, eqt1d.f_df_dpsi)
    f2 .= f2 .- f2[end] .+ fend ^ 2
    eqt1d.f = sign(fend) .* sqrt.(f2)

    pend = eqt1d.pressure[end]
    eqt1d.pressure = IMAS.cumtrapz(eqt1d.psi, eqt1d.dpressure_dpsi)
    eqt1d.pressure .+= pend .- eqt1d.pressure[end]

    eq2d.grid_type.index = 1
    eq2d.grid.dim1 = collect(range(canvas.Rs[1], canvas.Rs[end], Npsi))
    eq2d.grid.dim2 = collect(range(canvas.Zs[1], canvas.Zs[end], Npsi))
    FRESCO.update_interpolation!(canvas)
    eq2d.psi = [canvas._Ψitp(r, z) for r in eq2d.grid.dim1, z in eq2d.grid.dim2]

    fw = IMAS.first_wall(dd.wall)
    IMAS.flux_surfaces(eqt, fw.r, fw.z)

    return dd
end

In [ ]:
function VDE!(dd::IMAS.dd, canvas::FRESCO.Canvas, profile::FRESCO.CurrentProfile, 
              QI::QED.QED_state, build::QED.QED_build, Δt::Real, Nt::Int;
              tolerance::Real=1e-6, relax::Real=0.25, Nout::Int=200, Nin::Int=3,
              converge_hard::Bool=false)

    ps = Plots.Plot{Plots.GRBackend}[]
    Zs = zeros(Nt+1)
    Zs[1] = canvas.Zaxis
    eqt = dd.equilibrium.time_slice[]
    A0 = eqt.global_quantities.area
    pp0 = profile.pprime
    @showprogress for k in 1:Nt
    
        # Update canvas from QED structs
        canvas.Ip = QED.Ip(QI)
        for (k, coil) in enumerate(canvas.coils)
            VacuumFields.set_current!(coil, build.Ic[k] * VacuumFields.turns(coil))
        end

        #profile.pprime = x -> (2.0 * k / Nt) * pp0(x)

        # Perform GS solve
        k>1 && FRESCO.set_flux_at_coils!(canvas)
        success = true
        try
            status = FRESCO.solve!(canvas, profile, Nout, Nin; relax, tolerance, debug=false, control=:eddy, initialize_current=false, initialize_mutuals=(k==1))
            if converge_hard && (status != 0)
                success = false
            else
                update_from_fresco!(dd, canvas, profile)
            end
            if eqt.global_quantities.area / A0 < 0.1
                # plasma is too small. exit
                success = false
            end
        catch e
            # Something happened... often plasma got too small 
            display(e)
            success = false
        end
        
        if !success
            Zs[k+1:end] .= NaN
            break
        end
      
        Zs[k+1] = canvas.Zaxis
        push!(ps, plot(canvas, xrange=(0.75,2.75), yrange=(-2,2), colorbar=false))
    
        # Update QED structs
        QI = FUSE.qed_init_from_imas(dd.equilibrium.time_slice[], dd.core_profiles.profiles_1d[]; uniform_rho = 101)
        update_build!(build, canvas, dd)
        
        # Evolve coupled system
        QI = QED.evolve(QI, η0, build, Δt, 1000)
    end

    return ps, Zs
end

In [ ]:
Δt = 2e-3;
Nt = 200;
relax = 0.25;
tolerance = 1e-6;
Nout = 200;
Nin = 3;

dd = deepcopy(dd_save);
canvas = deepcopy(canvas_save);
VacuumFields.set_current!(canvas.coils[13], 1e3); # set upper VS coil current to 1 kA to perturb
QI = deepcopy(Q_save);
build = Build(canvas, dd);
ps, Zs = VDE!(dd, canvas, profile, QI, build, Δt, Nt; relax, tolerance, Nout, Nin, converge_hard=false);

In [ ]:
for (k,p) in enumerate(ps)
    plot!(p, title="t=$(round(1e3k*Δt)) ms", xlabel="", ylabel="", link=:both, xrange=(1,13),yrange=(-8,8))
end
k = length(ps)
plot!(ps[end], title="Final t=$(round(1e3k*Δt)) ms", xlabel="", ylabel="", link=:both, xrange=(1,13),yrange=(-8,8))

dN = Nt ÷ 10
if length(ps) == Nt
    display(plot(ps[dN:dN:end]..., layout=(2,5), size=(1300,700)))
else
    display(plot(vcat(ps[dN:dN:end], ps[end])..., layout=(2,5), size=(1300,700)))
end

In [ ]:
ts = range(0.0, Nt * Δt * 1000., Nt+1)
Zs[end-20:end] .= NaN
pz = plot(ts, Zs .- Zs[1], xlabel="t (ms)", ylabel="ΔZ (m)", lw=2, legend=nothing)
display(pz)
if true
    plot!(pz, yscale=:log10, ylims=(1e-2,3))
    gamma = 1 / 507
    fac = 4.2
    @show 1.0 / (fac * gamma)
    display(plot!(ts, exp.(-3.3 .+ fac.* gamma .* ts)))
end
#plot!(pz, ts, Zs65 .- Zs65[1], lw=2)